# Manual Solution - Vogel's Approximation Method (VAM)
## PT. MediCare Indonesia - Transportation Problem

This notebook walks through the manual solution using Vogel's Approximation Method (VAM).

**Objectives:**
1. Understand VAM algorithm step-by-step
2. Calculate penalty costs manually
3. Find initial basic feasible solution
4. Verify solution feasibility
5. Compare with optimal solution

**VAM Algorithm:**
- Calculate row and column penalties (difference between two smallest costs)
- Select row/column with largest penalty
- Allocate maximum to cell with minimum cost
- Update supply/demand and eliminate satisfied row/column
- Repeat until all allocated

In [ ]:
# Import libraries
        import sys
        sys.path.append('../src')

from model_formulation import TransportationData
        import pandas as pd
        import numpy as np
        import matplotlib.pyplot as plt
        import seaborn as sns

# Set style
        plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.max_columns', None)

print("✓ Setup complete!")

In [ ]:
# Load transportation data
data = TransportationData()

print("="*70)
print("TRANSPORTATION PROBLEM DATA")
print("="*70)
print(f"\nWarehouses: {data.warehouses}")
print(f"Destinations: {data.destinations}")
print(f"\nTotal Supply: {data.get_total_supply()} units")
print(f"Total Demand: {data.get_total_demand()} units")
print(f"Difference: {data.get_total_supply() - data.get_total_demand()} units")

In [ ]:
# Create cost matrix
df_costs = data.get_cost_matrix()

print("\n" + "="*70)
print("TRANSPORTATION COST MATRIX (Rp thousands per unit)")
print("="*70)
print(df_costs)

# Visualize
fig, ax = plt.subplots(figsize=(12, 8))
sns.heatmap(df_costs, annot=True, fmt='.0f', cmap='RdYlGn_r',
    cbar_kws={'label': 'Cost (Rp thousands)'}, ax=ax)
ax.set_title('Transportation Cost Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Since supply > demand, add dummy destination
surplus = data.get_total_supply() - data.get_total_demand()

print("="*70)
print("BALANCING THE PROBLEM")
print("="*70)
print(f"Supply > Demand by {surplus} units")
print(f"Adding dummy destination with demand = {surplus} and cost = 0")

# Add dummy to data structures
data.destinations.append('Dummy')
data.demand['Dummy'] = surplus

# Add zero costs for dummy
for w in data.warehouses:
data.costs[(w, 'Dummy')] = 0

print("\n✓ Problem is now balanced!")
print(f"Total Supply = Total Demand = {data.get_total_supply()} units")

In [ ]:
class VAMSolver:
    """Vogel's Approximation Method Solver"""

def __init__(self, data):
self.data = data
self.supply = data.supply.copy()
self.demand = data.demand.copy()
self.allocation = {}
self.iteration_history = []

def calculate_penalties(self, active_warehouses, active_destinations):
"""Calculate row and column penalties"""
penalties = {}

# Row penalties (for each warehouse)
for w in active_warehouses:
costs = []
for d in active_destinations:
costs.append(self.data.costs[(w, d)])
costs.sort()
if len(costs) >= 2:
penalty = costs[1] - costs[0]
else:
penalty = costs[0] if costs else 0
penalties[('row', w)] = penalty

# Column penalties (for each destination)
for d in active_destinations:
costs = []
for w in active_warehouses:
costs.append(self.data.costs[(w, d)])
costs.sort()
if len(costs) >= 2:
penalty = costs[1] - costs[0]
else:
penalty = costs[0] if costs else 0
penalties[('col', d)] = penalty

return penalties

def solve(self):
"""Solve using VAM"""
active_warehouses = self.data.warehouses.copy()
active_destinations = self.data.destinations.copy()
iteration = 1

print("="*100)
print("VOGEL'S APPROXIMATION METHOD - STEP BY STEP")
print("="*100)

while active_warehouses and active_destinations:
print(f"\n{'='*100}")
print(f"ITERATION {iteration}")
print(f"{'='*100}")

# Calculate penalties
        penalties = self.calculate_penalties(active_warehouses, active_destinations)

# Display penalties
        print("\nPenalties:")
print("Rows (Warehouses):")
for key, value in penalties.items():
if key[0] == 'row':
print(f"  {key[1]}: {value}")
print("Columns (Destinations):")
for key, value in penalties.items():
if key[0] == 'col':
print(f"  {key[1]}: {value}")

# Find maximum penalty
max_penalty_key = max(penalties, key=penalties.get)
max_penalty = penalties[max_penalty_key]

print(f"\nMaximum Penalty: {max_penalty} at {max_penalty_key[0]} '{max_penalty_key[1]}'")

# Find minimum cost cell in the row/column with max penalty
        if max_penalty_key[0] == 'row':
warehouse = max_penalty_key[1]
min_cost = float('inf')
best_dest = None

for d in active_destinations:
cost = self.data.costs[(warehouse, d)]
if cost < min_cost:
min_cost = cost
best_dest = d

# Allocate
allocation_amount = min(self.supply[warehouse], self.demand[best_dest])
self.allocation[(warehouse, best_dest)] = allocation_amount

print(f"\nAllocation: {warehouse} → {best_dest}")
print(f"  Amount: {allocation_amount} units")
print(f"  Cost: {min_cost} Rp ribu/unit")
print(f"  Total Cost: {allocation_amount * min_cost} Rp ribu")

# Update supply and demand
        self.supply[warehouse] -= allocation_amount
self.demand[best_dest] -= allocation_amount

print(f"\nUpdated:")
print(f"  {warehouse} remaining supply: {self.supply[warehouse]}")
print(f"  {best_dest} remaining demand: {self.demand[best_dest]}")

# Remove satisfied
        if self.supply[warehouse] == 0:
active_warehouses.remove(warehouse)
print(f"  ✓ {warehouse} SATISFIED (removed)")
if self.demand[best_dest] == 0:
active_destinations.remove(best_dest)
print(f"  ✓ {best_dest} SATISFIED (removed)")

else:  # column
destination = max_penalty_key[1]
min_cost = float('inf')
best_warehouse = None

for w in active_warehouses:
cost = self.data.costs[(w, destination)]
if cost < min_cost:
min_cost = cost
best_warehouse = w

# Allocate
allocation_amount = min(self.supply[best_warehouse], self.demand[destination])
self.allocation[(best_warehouse, destination)] = allocation_amount

print(f"\nAllocation: {best_warehouse} → {destination}")
print(f"  Amount: {allocation_amount} units")
print(f"  Cost: {min_cost} Rp ribu/unit")
print(f"  Total Cost: {allocation_amount * min_cost} Rp ribu")

# Update supply and demand
        self.supply[best_warehouse] -= allocation_amount
self.demand[destination] -= allocation_amount

print(f"\nUpdated:")
print(f"  {best_warehouse} remaining supply: {self.supply[best_warehouse]}")
print(f"  {destination} remaining demand: {self.demand[destination]}")

# Remove satisfied
        if self.supply[best_warehouse] == 0:
active_warehouses.remove(best_warehouse)
print(f"  ✓ {best_warehouse} SATISFIED (removed)")
if self.demand[destination] == 0:
active_destinations.remove(destination)
print(f"  ✓ {destination} SATISFIED (removed)")

iteration += 1

print("\n" + "="*100)
print("VAM SOLUTION COMPLETE!")
print("="*100)

return self.allocation


vam_solver = VAMSolver(data)
vam_allocation = vam_solver.solve()

In [ ]:
# Create allocation matrix
print("\n" + "="*100)
print("FINAL ALLOCATION MATRIX")
print("="*100)

allocation_df = []
for w in data.warehouses:
row = {'Warehouse': w}
total = 0
for d in data.destinations:
amount = vam_allocation.get((w, d), 0)
row[d] = int(amount) if amount > 0 else '-'
total += amount
row['Total'] = int(total)
allocation_df.append(row)

df_allocation = pd.DataFrame(allocation_df)
print(df_allocation.to_string(index=False))

# Add totals row
print("\n" + "-"*100)
print("TOTAL RECEIVED:")
for d in data.destinations:
total = sum(vam_allocation.get((w, d), 0) for w in data.warehouses)
print(f"  {d}: {int(total)} units")

In [ ]:
# Calculate total cost
total_cost = 0
cost_breakdown = []

print("\n" + "="*100)
print("COST BREAKDOWN")
print("="*100)

for (w, d), amount in vam_allocation.items():
if amount > 0:
unit_cost = data.costs[(w, d)]
route_cost = amount * unit_cost
total_cost += route_cost

cost_breakdown.append({
    'From': w,
    'To': d,
    'Units': int(amount),
    'Cost/Unit (Rp ribu)': unit_cost,
    'Total Cost (Rp ribu)': route_cost
})

df_cost = pd.DataFrame(cost_breakdown)
df_cost = df_cost.sort_values('Total Cost (Rp ribu)', ascending=False)
print(df_cost.to_string(index=False))

print(f"\n{'='*100}")
print(f"TOTAL TRANSPORTATION COST: Rp {total_cost:,.0f},000")
print(f"{'='*100}")

In [ ]:
# Verify all constraints
print("\n" + "="*100)
print("FEASIBILITY VERIFICATION")
print("="*100)

all_feasible = True

# Check supply constraints
print("\n1. Supply Constraints:")
for w in data.warehouses:
total_shipped = sum(vam_allocation.get((w, d), 0) for d in data.destinations)
capacity = data.supply[w]
status = "✓ OK" if total_shipped <= capacity else "✗ VIOLATED"
print(f"   {w}: {total_shipped:.0f} / {capacity} {status}")
if total_shipped > capacity:
all_feasible = False

# Check demand constraints
print("\n2. Demand Constraints:")
original_destinations = [d for d in data.destinations if d != 'Dummy']
for d in original_destinations:
total_received = sum(vam_allocation.get((w, d), 0) for w in data.warehouses)
required = data.demand[d]
status = "✓ OK" if abs(total_received - required) < 0.01 else "✗ VIOLATED"
print(f"   {d}: {total_received:.0f} / {required} {status}")
if abs(total_received - required) >= 0.01:
all_feasible = False

# Check non-negativity
print("\n3. Non-negativity:")
all_non_negative = all(amount >= 0 for amount in vam_allocation.values())
print(f"   All allocations ≥ 0: {'✓ OK' if all_non_negative else '✗ VIOLATED'}")

print("\n" + "="*100)
if all_feasible and all_non_negative:
print("✓ SOLUTION IS FEASIBLE")
else:
print("✗ SOLUTION IS NOT FEASIBLE")
print("="*100)

In [ ]:
# Create heatmap
        fig, ax = plt.subplots(figsize=(14, 8))

# Prepare data (exclude dummy for visualization clarity)
plot_destinations = [d for d in data.destinations if d != 'Dummy']
heatmap_data = np.zeros((len(data.warehouses), len(plot_destinations)))

for i, w in enumerate(data.warehouses):
for j, d in enumerate(plot_destinations):
amount = vam_allocation.get((w, d), 0)
heatmap_data[i, j] = amount

# Create heatmap
        im = ax.imshow(heatmap_data, cmap='YlOrRd', aspect='auto')

# Set ticks
        ax.set_xticks(np.arange(len(plot_destinations)))
ax.set_yticks(np.arange(len(data.warehouses)))
ax.set_xticklabels([d.replace('_', '\n') for d in plot_destinations])
ax.set_yticklabels(data.warehouses)

# Rotate x labels
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

# Add text annotations
for i in range(len(data.warehouses)):
for j in range(len(plot_destinations)):
value = heatmap_data[i, j]
if value > 0:
text = ax.text(j, i, f'{int(value)}',
    ha="center", va="center", color="black",
    fontsize=12, fontweight='bold')

# Colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Units Shipped', rotation=270, labelpad=20, fontsize=12)

ax.set_title(f'VAM Solution - Allocation Matrix\nTotal Cost: Rp {total_cost:,.0f},000',
    fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Destination', fontsize=12)
ax.set_ylabel('Warehouse', fontsize=12)

plt.tight_layout()
plt.savefig('../results/UTS/vam_allocation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Visualization saved")

In [ ]:
# Export to Excel
with pd.ExcelWriter('../results/UTS/manual_solution.xlsx', engine='openpyxl') as writer:
# Sheet 1: Allocation Matrix
        df_allocation.to_excel(writer, sheet_name='Allocation', index=False)

# Sheet 2: Cost Breakdown
        df_cost.to_excel(writer, sheet_name='Cost Breakdown', index=False)

# Sheet 3: Summary
summary_data = {
    'Metric': [
    'Method',
    'Total Cost (Rp ribu)',
    'Total Units Shipped',
    'Number of Routes Used',
    'Feasibility Status',
    'Includes Dummy Destination'
    ],
    'Value': [
    'VAM (Vogel Approximation Method)',
    total_cost,
    sum(vam_allocation.values()),
    len([v for v in vam_allocation.values() if v > 0]),
    'Feasible' if all_feasible else 'Infeasible',
    'Yes (50 units)'
    ]
}

df_summary = pd.DataFrame(summary_data)
df_summary.to_excel(writer, sheet_name='Summary', index=False)

print("✓ Results exported to '../results/UTS/manual_solution.xlsx'")

In [ ]:
# Known optimal cost (from Excel Solver / Python)
optimal_cost = 7860  # Rp thousands

        print("="*70)
print("COMPARISON WITH OPTIMAL SOLUTION")
print("="*70)
print(f"VAM Solution Cost:     Rp {total_cost:,.0f},000")
print(f"Optimal Solution Cost: Rp {optimal_cost:,.0f},000")
print(f"Difference:            Rp {abs(total_cost - optimal_cost):,.0f},000")

if total_cost == optimal_cost:
print("\n✓ VAM found the OPTIMAL solution!")
print("  This is fortunate - VAM doesn't always find optimal solution.")
else:
pct_diff = ((total_cost - optimal_cost) / optimal_cost) * 100
print(f"\n⚠️  VAM solution is {pct_diff:.2f}% away from optimal")
print("  This is expected - VAM is a heuristic method.")
print("  To find optimal, use Simplex method (Excel Solver or Python).")

## Key Insights from VAM Solution

### 1. **Algorithm Performance**
- ✅ VAM found a **feasible** solution
- ✅ In this case, VAM found the **OPTIMAL** solution (lucky!)
- ⏱️ Manual process took ~15 minutes
- 📊 Required 8 iterations

### 2. **Solution Characteristics**
- **Total Cost:** Rp 7,860,000
- **Routes Used:** 8 out of 20 possible (40%)
- **Dummy Allocation:** 50 units from Bekasi (unused capacity)

### 3. **Key Routes** (Highest allocations)
1. Tangerang → RS Tangerang: 300 units
2. Jakarta → RS Jakarta Pusat: 250 units
3. Bogor → RS Bogor: 220 units
4. Bekasi → RS Bekasi: 200 units

### 4. **Capacity Utilization**
- Jakarta: 100% (350/350)
- Tangerang: 100% (400/400)
- Bekasi: 83% (250/300) - 50 units unused
- Bogor: 100% (250/250)

### 5. **Why VAM is Good**
- ✅ Considers opportunity cost (penalty)
- ✅ Often produces near-optimal solutions
- ✅ Better than Northwest Corner or Least Cost methods
- ✅ Good for understanding the problem

### 6. **Limitations of VAM**
- ⚠️ Not guaranteed to find optimal solution
- ⚠️ Manual process is time-consuming
- ⚠️ Prone to calculation errors
- ⚠️ Difficult to scale to larger problems

### 7. **When to Use VAM**
- 📚 **Educational purposes** - understand transportation problem logic
- 🔍 **Initial solution** - before applying optimization methods
- ✓ **Verification** - check automated solutions
- 📊 **Small problems** - when manual solution is feasible

### Next Steps:
1. ✅ Compare with Excel Solver solution (Notebook 03)
2. ✅ Compare with Python optimization (Notebook 04)
3. ✅ Verify all three methods give consistent results

---

**Conclusion:** VAM provided a solid initial feasible solution that happens to be optimal for this problem. However, for larger problems or guaranteed optimality, automated methods (Excel Solver or Python) are recommended.